In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1422, with_units=False)
Hist_table


Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"10s 29 ; 25,10"
730 ;,"09s 28 ; 50,21"
1095 ;,"08s 28 ; 15,31"
1461 ;,"07s 28 ; 34,51"
1826 ;,"06s 28 ; 00,01"
2191 ;,"05s 27 ; 25,11"
2556 ;,"04s 26 ; 50,22"
2922 ;,"03s 27 ; 09,41"
3287 ;,"02s 26 ; 34,51"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(34):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),3)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),3)

In [4]:
# moon mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("00;54,09,04,10,11,50,56"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(34):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=6):
        days=Hist_table[i][0]
        Test_recomp[i][1]=(daily_mean*days)%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=3600*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Years,Entries
IntegerAndSexagesimal,Historical
365 ;,"10s 29 ; 25,10"
730 ;,"09s 28 ; 50,21"
1095 ;,"08s 28 ; 15,31"
1461 ;,"07s 28 ; 34,51"
1826 ;,"06s 28 ; 00,01"
2191 ;,"05s 27 ; 25,11"
2556 ;,"04s 26 ; 50,22"
2922 ;,"03s 27 ; 09,41"
3287 ;,"02s 26 ; 34,51"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(-1076.4705882352941, nan)

In [6]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Years,
365 ;,0.000000
730 ;,0.000000
1095 ;,0.000000
1461 ;,0.000000
1826 ;,0.000000
2191 ;,0.000000
2556 ;,0.000000
2922 ;,0.000000
3287 ;,0.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Jupiter mean arg years.csv')
B.to_csv('Jupiter mean arg years.zip', index=False, compression=compression_opts)